<a href="https://colab.research.google.com/github/m-umamaheshwari/Job-match-score/blob/main/Job_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers peft gradio pymupdf torch


In [ ]:
import gradio as gr
from transformers import AutoModel, AutoTokenizer
from peft import PeftModel
import torch
import torch.nn.functional as F
import fitz

# 🧠 Load the model + tokenizer
base_model = AutoModel.from_pretrained("BAAI/bge-large-en-v1.5")
model = PeftModel.from_pretrained(base_model, "shashu2325/resume-job-matcher-lora")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en-v1.5")

# 📄 Text extractor from PDF
def extract_text(file_obj):
    try:
        with fitz.open(file_obj.name) as doc:
            return "\n".join([page.get_text() for page in doc])
    except Exception as e:
        return f"❌ Error reading PDF: {e}"

# 💡 Calculate similarity score
def compute_similarity(resume_text, job_text):
    resume_inputs = tokenizer(resume_text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)
    job_inputs = tokenizer(job_text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)

    with torch.no_grad():
        resume_outputs = model(**resume_inputs)
        job_outputs = model(**job_inputs)

        resume_emb = resume_outputs.last_hidden_state.mean(dim=1)
        job_emb = job_outputs.last_hidden_state.mean(dim=1)

        resume_emb = F.normalize(resume_emb, p=2, dim=1)
        job_emb = F.normalize(job_emb, p=2, dim=1)

        similarity = torch.sum(resume_emb * job_emb, dim=1)
        return torch.sigmoid(similarity).item() * 100

# 🎯 Gradio function
def analyze_resume(resume_file, job_file):
    resume_text = extract_text(resume_file)
    job_text = extract_text(job_file)

    if not resume_text.strip() or not job_text.strip():
        return "⚠️ Could not extract text from files."

    score = compute_similarity(resume_text, job_text)

    if score >= 80:
        color = "green"
        emoji = "🟢 Strong Match"
    elif score >= 60:
        color = "orange"
        emoji = "🟠 Moderate Match"
    else:
        color = "red"
        emoji = "🔴 Weak Match"

    styled = f"<h2 style='color:{color}; font-size: 26px;'>✅ Match Score: {score:.2f}/100<br>{emoji}</h2>"
    return styled

# 🚀 Gradio interface with sections, markdown, and colors
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
        <h1 style="text-align:center; color:#333;">📄 AI Resume Matcher</h1>
        <p style="text-align:center;">Upload your resume and a job description to check alignment using a fine-tuned AI model.</p>
    """)

    with gr.Row():
        resume_input = gr.File(label="📄 Upload Resume (PDF)", type="filepath", file_types=[".pdf"])
        jd_input = gr.File(label="📝 Upload Job Description (PDF)", type="filepath", file_types=[".pdf"])

    btn = gr.Button("✨ Analyze Match", variant="primary")

    output = gr.HTML(label="Result")

    btn.click(analyze_resume, inputs=[resume_input, jd_input], outputs=output)

    gr.Markdown("<hr><center><small>Made by Uma Maheshwari 💼 | Model: BGE LoRA Fine-tuned</small></center>")

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://24b6ee342c2128b5b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The cell `mYTEeXWMZBJt` previously contained only a URL string, which is not a valid Python command and resulted in a `SyntaxError`. If you intended to use this URL to interact with the Git repository, you should use it with the appropriate Git commands, such as `!git clone <URL>`.